In [332]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
#from selenium.webdriver.firefox.options import Options
import pandas as pd
import time
import ipywidgets as widgets
from numpy import *
from IPython.display import display_html
from itertools import chain, cycle

## Instructions

From the menu, select Cell -> Run All, and wait about 5-10 seconds.  If the application doesn't appear, try it again.  It sometimes takes two tries for me...

Hint:  If you see something like `In: [*]` on the left of a cell, the code is still running.  Wait until you see `In: []`, *.i.e.,* the asterisk dissappears to try again.

Once you see the application:
    You should see three tabs:  Individual Event (on top), Team, and Projection
    
    On the Individual Event tab:
        You can exclude athletes / groups of athletes from the scoring by selecting them in the "Omit athletes" box.  A single click will select one athlete.  To select more than one, hold <CTRL> while clicking.  These athletes will remain excluded from scoring until you deselet them (<CTRL>-click on selected name in Omit athletes) or you click on the "exclude none" button, which will remove all omitted athletes from ALL EVENTS.

In [333]:

options = webdriver.ChromeOptions()
#options = webdriver.FirefoxOptions()
#options = Options()
#options.headless = True
options.add_argument('--headless')

In [334]:
driver = webdriver.Chrome(options = options)
#driver = webdriver.Firefox(options = options)

In [335]:
#driver = webdriver.Chrome()

In [336]:
# connect to latest web page
driver.get("https://www.tfrrs.org/lists/4728/Northwest_Conference_Outdoor_Performance_List")
time.sleep(2.5)

In [337]:
elem = driver.find_element(By.NAME, 'limit')
elem2 = driver.find_element(By.NAME, 'event_type')
elem3 = driver.find_element(By.NAME, 'gender')
select = Select(elem)
select.select_by_index(3)  #as per https://github.com/SeleniumHQ/seleniumhq.github.io/blob/trunk//examples/python/tests/support/test_select_list.py#L9-L10
time.sleep(2)

In [339]:
# get list of table titles
titles = ['']
for i in range(5):
    #print("# iterations = %d" %i)
    titlelist = driver.find_elements(By.CLASS_NAME, "custom-table-title")
    try:
        titles = [title.text.split('\n')[0].split('NWC')[0].strip() for title in titlelist]
        if titles[0] != "":
            #print(titles[0])
            break
    except StaleElementReferenceException:
        pass
    time.sleep(1)
    
# read in tables using Pandas
tables = pd.read_html(driver.page_source)
tables = [table.rename(columns = {'Unnamed: 0':'Rank'}) for table in tables]

# make dictionary of data
title_dict = dict(zip(titles, tables))

# done with browser for now
del driver

In [340]:
# set up definitions of scoring 
team_names = "Whitworth:Linfield:Pacific Lutheran:George Fox:Pacific (Ore.):Lewis & Clark:Willamette:Puget Sound:Whitman".split(":")
rank_scores = [10, 8, 6, 5, 4, 3, 2, 1] + [0]*50

def score_event(event_key):
    '''Computes score for event AND adds (1-time event) columns for Score and Excluded'''
    
    df = title_dict[event_key]
    ranks = df['Rank'].to_list()
    scores = [ average( rank_scores[rank-1 : rank - 1 + ranks.count(rank) ] ) for rank in ranks]
    exclusions = [""] * len(ranks)
    try:
        df.insert(1, "Score", scores)
    except ValueError:
        pass
        
    try: 
        df.insert(0, "Excluded", exclusions)
    except ValueError:
        pass
    return df
    

def team_score_event(event = "High Jump (Women)" , team = "George Fox"):
    '''Calculates total by team for a given event'''
    
    df = title_dict[event]
    if 'Score' not in df:
        score_event(event)
    return df.loc[df['Team']==team, 'Score'].sum(), df.loc[df['Team']==team, 'Score'].to_list()

def create_team_event_table(event = "High Jump (Women)"):
    dd = {'Team':team_names, 'Score':[team_score_event(event=event, team= team)[0] for team in team_names]}
    df = pd.DataFrame(dd)
    return df

def team_total(team = "Whitworth", gender = "Men"):
    df = create_team_table(team, gender)
    return df['Total'].sum()

def create_team_table(team = "Whitworth", gender = "Men"):
    event_list = [title for title in titles if gender in title]
    totals = []
    scores = []
    events = []
    for event in event_list:
        total, score = team_score_event(event, team)
        totals.append(total)
        result = str([item for item in score if item]) if sum(score)>0 else ""
        scores.append(result)
        events.append(event)
    dd = {'Total': totals, 'Event': events, 'Score(s)': scores}
    return pd.DataFrame(dd).sort_values(by=['Total'], ascending=False)

def create_projection_table(gender = "Men"):
    points = [ team_total(team = team, gender = gender) for team in team_names]
    place = list(range(1, 10))
    dd = {"Men's Team": team_names, "Points": points}
    df = pd.DataFrame(dd).sort_values(by=['Points'], ascending=False)
    df.insert(0, 'Place', place)
    return df

In [341]:
# from https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2 style="text-align: center;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [342]:
#  add columns for scores and exclude

for event in titles:
    score_event(event)
    
    

In [343]:
# construct ipywidget widgets for controls
drop_event = widgets.Dropdown(options = titles, value = titles[0])
drop_athlete = widgets.SelectMultiple(options = ['none'], value = ['none'], description = "Omit athletes",
                                     continuous_update = False)
reset_button = widgets.Button(description = "exclude none", button_style = "success")
outbox = widgets.Output(layout = {'border': '1px solid black'})
# errbox is used to print to in order to see what is happening; outbox gets cleared regularly
errbox = widgets.Output(layout = {'border': '1px solid red'})

# second tab widgets
drop_team = widgets.Dropdown(options = team_names, value = "Whitworth")
drop_gender = widgets.Dropdown(options = ['Men', 'Women'], value = 'Men')
hbox2 = widgets.HBox([drop_team, drop_gender])
outbox2 = widgets.Output(layout = {'border': '1px solid black'})
vbox2 = widgets.VBox([hbox2, outbox2])
place_holder = widgets.Label("This is a placeholder")

# third tab
outbox3 = widgets.Output(layout = {'border': '1px solid blue'})
#drop_event

def exclude_none(b):
    '''Removes all exclusions from all events'''
    with errbox:
        print("button clicked")
    for event, table in title_dict.items():
        excl = table['Excluded'].to_list()
        if 'X' in excl:
            with errbox:
                print("found an exclusion in %s" %event)
                print(table['Excluded'])
                print("updated to")
                table['Excluded'] = ''
                print(table['Excluded'])
                print()
            
            rescore_event(event)
    reset_result("from exclude_none")
    
def update_exclusions(change):
    '''Called by changing exclusions interactively'''
    
    exclusion_list = drop_athlete.value
    with errbox:
        print("updating_exclusions to", exclusion_list)
    event = drop_event.value
    change_exclusions(event, exclusion_list)
    reset_result('updating exclusions')
    
def change_exclusions(event, exclusion_list=[]):
    '''Can call for an event with a list of athletes to exclude or with an empty
        list in order to clear exclusions'''
    
    df = title_dict[event]
    if exclusion_list:
        ath_key = 'Athlete' if 'Athlete' in df else 'Athletes'
        mask = df[ath_key].isin(exclusion_list)
        df['Excluded'] = ''
        df.loc[mask, 'Excluded'] = 'X'
    else:
        df['Excluded'] = ''
    
    
def rescore_event(event):
    '''Recalculate scores based on any excluded athletes'''
    
    with errbox:
        print("rescore_event called")
        try:
            print(event)
        except TypeError:
            print("ERROR")
    event_key = event
    df = title_dict[event_key]
    mask = df['Excluded'] == 'X'
    ranks = df.loc[~mask, 'Rank'].to_list()
    scores = [ average( rank_scores[ranks.index(rank) : ranks.index(rank)  + ranks.count(rank) ] ) for rank in ranks]
    df.loc[mask, 'Score'] = 0
    df.loc[~mask, 'Score'] = scores
    
def reset_result(change):
    '''update all viewable results'''
    
    with errbox:
        print("reset_result called from")
        try:
            print("\t",change['old'])
        except TypeError:
            print("\t",change)
    outbox.clear_output()
    event = drop_event.value
    rescore_event(event)
    df = title_dict[event]
    drop_athlete.unobserve(update_exclusions, names = 'value')
    ath_key = 'Athlete' if 'Athlete' in df else 'Athletes'
    drop_athlete.options = df[ath_key].to_list()
    excluded = df.loc[df['Excluded']=='X',ath_key].to_list()
    #print("excluded = ", excluded)
    drop_athlete.value = excluded
    drop_athlete.observe(update_exclusions, names = 'value')
    
    with outbox:
        team_df = create_team_event_table(event)
        team_df = team_df.sort_values(by=['Score'], ascending=False)
       
        display_side_by_side(df.loc[:, df.columns != 'Meet'].iloc[:12].style.hide(axis='index').format(precision=2),
                             team_df.style.hide(axis="index").format(precision=2),
                             titles = ["%s Results"% event, "Team Totals"])
            
    team_result('reset')
    update_projections('reset')
        
def team_result(change):
    outbox2.clear_output()
    team = drop_team.value
    gender = drop_gender.value
    df = create_team_table(team, gender)
    total = df['Total'].sum()
    with outbox2:
        title = team + " " + gender + " " + "%.2f" % total
        display_side_by_side(df.style.hide(axis="index").format(precision=2), titles=[title])
        
def update_projections(change):
    outbox3.clear_output()
    df = create_projection_table()
    sum_men = df['Points'].sum()
    df2 = create_projection_table(gender="Women")
    sum_women = df2['Points'].sum()
    with outbox3:
        display_side_by_side(df.style.hide(axis="index").format(precision=2), df2.style.hide(axis="index").format(precision=2), 
                             titles = ["Men %.1f"%sum_men, "Women %.1f"%sum_women])
    
drop_event.observe(reset_result, names = 'value')
drop_athlete.observe(update_exclusions, names = 'value')
drop_team.observe(team_result)
drop_gender.observe(team_result)
reset_button.on_click(exclude_none)

hbox = widgets.HBox([drop_event, drop_athlete, reset_button])
# for error message:
#box = widgets.VBox([hbox, outbox, errbox])
# clean version
box = widgets.VBox([hbox, outbox])

reset_result('arf')

tab = widgets.Tab(children = [box, vbox2, outbox3], titles = ['Individual Event', 'Team', 'Projection'])
tab

In [ ]:
%%html
<style>
div.input{
    display:none;
}
</style>